# MAIN CODE

In [1]:
import os
import pandas as pd
import datetime
import re

# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def parse_chat_file(file_path, expected_date):
    chat_data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date:
                continue

            # Adjusted logic to check for numeric sender (phone number)
            is_person = re.match(r'^[+\d\s-]+$', sender) is None  # True if sender is NOT purely numeric

            # Debug: Print sender and is_person flag
            print(f"Sender: {sender}, Is Person: {is_person}")

            chat_data.append((date_time, sender, is_person))
    return chat_data

# Function to create a template dataframe
def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=intervals)
    return df

# Function to populate the dataframe
def populate_dataframe(df, parsed_data, start_column_index):
    person_column_index = start_column_index
    for entry in parsed_data:
        date_time, sender, is_person = entry
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        if person_column_index not in df.columns:
            df[person_column_index] = 0
        if person_column_index + 1 not in df.columns:
            df[person_column_index + 1] = 0

        if is_person:
            df.at[interval, person_column_index] = 1
        else:
            df.at[interval, person_column_index + 1] = 1

    return person_column_index + 2

def process_person_chats(chat_files):
    dataframes = {}
    for file in chat_files:
        parts = file.split(os.sep)
        date_folder, person = parts[-4], parts[-2]

        try:
            expected_date = pd.to_datetime(date_folder).date()
        except ValueError:
            print(f"Skipping file due to incorrect date format in folder name: {file}")
            continue

        key = f"{expected_date.strftime('%Y-%m-%d')}_{person}"

        if key not in dataframes:
            dataframes[key] = create_template_dataframe()
            start_column_index = 0
        else:
            # Check if the dataframe already has columns
            if not dataframes[key].columns.empty:
                start_column_index = max(dataframes[key].columns) + 1
            else:
                start_column_index = 0

        parsed_data = parse_chat_file(file, expected_date)
        next_column_index = populate_dataframe(dataframes[key], parsed_data, start_column_index)
    
    return dataframes




# Main script
date_directory = "C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Test\\filtered_chats"
chat_files = list_chat_files(date_directory)
person_dataframes = process_person_chats(chat_files)

# Save each dataframe as a CSV file in the current working directory for testing purpose only, will be omiited in real application



Sender: +263 77 430 1947, Is Person: False
Sender: +263 77 430 1947, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: +263 78 501 9780, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: +263 78 501 9780, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: +263 78 501 9780, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: +263 78 501 9780, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: +263 78 501 9780, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: +263 78 501 9780, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: +263 78 501 9780, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: +263 78 501 9780, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: +263 78 501 9780, Is Person: F

Sender: +263 77 229 7277, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: +263 77 229 7277, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: +263 77 229 7277, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: +263 77 229 7277, Is Person: False
Sender: +263 77 229 7277, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: +263 77 229 7277, Is Person: False
Sender: +263 77 229 7277, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: +263 77 229 7277, Is Person: False
Sender: +263 77 229 7277, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: +263 77 229 7277, Is Person: False
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: +263 77 229 7277, Is Person: F

Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: Aditi Edoofa, Is Person: True
Sender: Jasmine Edoofa, Is Person: True
Sender: +263 78 125 6089, Is Person: False
Sender: Jasmine Edoofa, Is Person: True
Sender: +263 78 125 6089, Is Person: False
Sender: Jasmine Edoofa, Is Person: True
Sender: +263 78 125 6089, Is Person: False
Sender: Jasmine Edoofa, Is Person: True
Sender: +263 78 125 6089, Is Person: False
Sender: +263 77 482 2511, Is Person: False
Sender: +263 77 482 2511, Is Person: False
Sender: Jasmine Edoofa, Is Person: True
Sender: +263 71 380 7287, Is Person: False
Sender: +263 71 380 7287, Is Person: False
Sender: +263 71 380 7287, Is Person: False
Sender: Jasmine Edoofa, Is Person: True
Sender: +263 71 380 7287, Is Person: False
Sender: Jasmine Edoofa, Is Person: True
Sender: Jasmine Edoofa, Is Person: True
Sender: +263 71 380 7287, Is Person: False
Sender: Jasmine Edoofa, Is Person: True
Sender: Jasmine

Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: +263 71 326 9377, Is Person: False
Sender: Saloni Edoofa, Is Person: True
Sender: +263 71 326 9377, Is Person: False
Sender: Saloni Edoofa, Is Person: True
Sender: +263 71 326 9377, Is Person: False
Sender: Saloni Edoofa, Is Person: True
Sender: +263 71 326 9377, Is Person: False
Sender: Saloni Edoofa, Is Person: True
Sender: +263 71 326 9377, Is Person: False
Sender: +263 71 326 9377, Is Person: False
Sender: Saloni Edoofa, Is Person: True
Sender: +263 71 326 9377, Is Person: False
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: +263 71 326 9377, Is Person: False
Sender: +263 71 326 9377, Is Person: False
Sender: +263 71 326 9377, Is Person: False
Sender: Saloni Edoofa, Is Person: True
Sender: +263 71 326 9377

Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: +263 77 887 4090, Is Person: False
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: +263 77 887 4090, Is Person: False
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Edoofa, Is Person: True
Sender: Saloni Ed

...

Sender: +263 77 448 2658, Is Person: False
Sender: Ashi Edoofa, Is Person: True
Sender: +263 77 448 2658, Is Person: False
Sender: +263 77 448 2658, Is Person: False
Sender: Ashi Edoofa, Is Person: True
Sender: +263 77 448 2658, Is Person: False
Sender: +265 885 25 22 54, Is Person: False
Sender: +265 885 25 22 54, Is Person: False
Sender: +265 992 34 99 23, Is Person: False
Sender: +265 992 34 99 23, Is Person: False
Sender: +263 77 503 3000, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +263 77 503 3000, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: Kirti Edoofa, Is Person: True
Sender: Kirti Edoofa, Is Person: True
Sender: Kirti Edoofa, Is Person: True
Sender: +263 77 503 3000, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +263 77 503 3000, Is Person: False
Sender: +263 77 503 3000, Is Person: False
Sender: +263 77 503 3000, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +263 77 503 3000, Is Person: False
Sender: Kir

Sender: Kirti Edoofa, Is Person: True
Sender: Kirti Edoofa, Is Person: True
Sender: Kirti Edoofa, Is Person: True
Sender: Kirti Edoofa, Is Person: True
Sender: Kirti Edoofa, Is Person: True
Sender: +260 763133583, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +260 763133583, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +260 763133583, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +260 763133583, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +260 763133583, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +263 71 470 8325, Is Person: False
Sender: +263 71 470 8325, Is Person: False
Sender: +263 77 979 7389, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +263 71 470 8325, Is Person: False
Sender: +263 71 470 8325, Is Person: False
Sender: +263 71 470 8325, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: Kirti Edoofa, Is Person: True
Sender: Kirti Edoofa, Is Person: True
Sende

Sender: Kirti Edoofa, Is Person: True
Sender: +263 78 203 3411, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +263 78 203 3411, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +263 78 203 3411, Is Person: False
Sender: Kirti Edoofa, Is Person: True
Sender: +263 78 203 3411, Is Person: False
Sender: +263 71 455 7432, Is Person: False
Sender: +263 77 540 1325, Is Person: False
Sender: +263 77 540 1325, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 77 540 1325, Is Person: False
Sender: +263 77 540 1325, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 77 448 6799, Is Person: False
Sender: +263 77 448 6799, Is Person: False
Sender: +263 77 733 5006, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 77 448 6799, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 77 448 6799, Is Person: False


Sender: +263 78 867 1027, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 78 867 1027, Is Person: False
Sender: +263 78 867 1027, Is Person: False
Sender: +263 78 867 1027, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 78 867 1027, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 78 867 1027, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 78 867 1027, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 78 867 1027, Is Person: False
Sender: +263 78 867 1027, Is Person: False
Sender: +263 78 867 1027, Is Person: False
Sender: +263 78 867 1027, Is Person: False
Sender: +263 78 867 1027, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 78 867 1027, Is Pers

Sender: +263 71 380 7287, Is Person: False
Sender: +263 71 380 7287, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 71 380 7287, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 71 380 7287, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 71 380 7287, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 71 380 7287, Is Person: False
Sender: +263 71 380 7287, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 78 706 9627, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 78 706 9627, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 78 706 9627, Is Person: False
Sender: Shivjeet Edoofa, Is Person: True
Sender: +263 78 706 9627, Is Person: False
Sender: Shivjeet Edoofa, Is Person:

In [2]:
for key, df in person_dataframes.items():
    csv_file_path = f"csvs/{key}.csv"
    df.to_csv(csv_file_path)
    print(f"Saved DataFrame to {csv_file_path}")

Saved DataFrame to csvs/2023-11-22_Aditi_Edoofa.csv
Saved DataFrame to csvs/2023-11-22_Jasmine_Edoofa.csv
Saved DataFrame to csvs/2023-11-22_Saloni_Edoofa.csv
Saved DataFrame to csvs/2023-11-22_Sharda_Edoofa.csv
Saved DataFrame to csvs/2023-11-22_Ashi_Edoofa.csv
Saved DataFrame to csvs/2023-11-22_Kirti_Edoofa.csv
Saved DataFrame to csvs/2023-11-22_Milan_Edoofa.csv
Saved DataFrame to csvs/2023-11-22_Shivjeet_Edoofa.csv


# DEBUGGING

In [ ]:
import os
import pandas as pd
import datetime
import re

# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files
    print("chats filed!")


In [ ]:
def parse_chat_file(file_path, expected_date):
    chat_data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # [existing regex pattern matching]

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date:
                continue

            # Check if sender's name is not purely numeric
            is_person = not sender.isnumeric() if sender else False

            # Debug: Print sender and is_person flag
            print(f"Sender: {sender}, Is Person: {is_person}")

            chat_data.append((date_time, sender, is_person))
    return chat_data


In [ ]:
# Example usage
sample_chat_file = chat_files[0]  # Assuming chat_files has been populated
expected_date = pd.to_datetime("2023-11-22").date()  # Replace with the appropriate date
parse_chat_file(sample_chat_file, expected_date)


In [3]:
pip install matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.
